In this notebook, we retrieve all the other structured data we need for our analysis and save it to disk for easy retrieval in subsequent notebooks. Once again, the OMOP tables are assumed to be accessible from a `dask-sql` context, as in `01-retrieve-population.ipynb`.

In [ ]:
# imports
import pandas as pd

# load cohort
cohort = pd.read_feather("<cohort filepath>")
person_ids = ", ".join(cohort["<person ID>"].astype(str).tolist())

# save table with patient information
query = c.sql(f"""
    SELECT *
    FROM <person table>
    WHERE <person ID> IN ({person_ids})
    ORDER BY <person ID>
""")
person_table = query.compute()
person_table.reset_index(drop=True).to_feather("<person table filepath>")

# save table with condition information
query = c.sql(f"""
    SELECT *
    FROM <condition occurrence table>
    WHERE <person ID> IN ({person_ids})
    ORDER BY <person ID>
""")
condition_table = query.compute()
condition_table.reset_index(drop=True).to_feather("<condition table filepath>")

# save table with concept information
concept_ids = sorted(condition_table["<condition concept ID>"].unique())
query = c.sql(f"""
    SELECT *
    FROM <concept table>
    WHERE <concept ID> IN ({", ".join(str(i) for i in concept_ids)})
""")
concept_table = query.compute()
concept_table.reset_index(drop=True).to_feather("<concept table filepath>")